In [ ]:
import pandas as pd
import os
import dalex as dx
from keras.models import load_model
from sklearn.utils import shuffle
from sklearn_extra.cluster import KMedoids
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
def read_Dataset(): # To read the Original Dataset

    path = './CICMalDroid2020/'
    train_dataset  =  pd.read_csv(path +'train.csv')
    test_dataset = pd.read_csv( path + 'test.csv')
    y_train = train_dataset['Class']
    y_test = test_dataset['Class']
    try:
        train_dataset.drop(['Class'], axis = 1, inplace=True)
    except IOError:
        print('IOERROR')
    try:
        test_dataset.drop(['Class'] , axis =1 , inplace= True)
    except IOError:
        print('IOERROR')

    print('Train Dataset shape : ', train_dataset.shape)
    print('Test Dataset shape : ', test_dataset.shape)
    print('YTrain Dataset shape : ', y_train.shape)
    print('Ytest Dataset shape : ', y_test.shape)
    
    print('y_train val_counts : ', y_train.value_counts())
    print('y_test val_counts : ', y_test.value_counts())

    return train_dataset, test_dataset, y_train,y_test


def read_T_A_Datasets(train_dataset,y_train,dataset_no): # To read the Adversarial Samples and Add them to the 
    #original Dataset

    path = './T_A/Datasets/'
    
    
    columns = train_dataset.columns

    train_adv_samples  =  pd.read_csv(path +'T_A_Dataset_MalDroid'+str(dataset_no)+'.csv')
    
    
    y_train_adv = train_adv_samples['Class']

    try:
        train_adv_samples.drop(['Class'], axis = 1, inplace=True)
    except IOError:
        print('IOERROR')

    print('Train Adv Dataset shape : ', train_adv_samples.shape)
    print('YTrain Adv Dataset shape : ', y_train_adv.shape)
    print('y_train adv val_counts : ', y_train_adv.value_counts())

    train_dataset = train_dataset.append(train_adv_samples)
    y_train = y_train.append(y_train_adv)

    print('\nTrain Dataset+Adversarial Samples shape : ', train_dataset.shape)
    print('\nYTrain Dataset+Adversarial Samples shape : ', y_train.shape)

    train_dataset,y_train = shuffle(train_dataset,y_train, random_state=42)
    
    
    return train_dataset, y_train

train_dataset, test_dataset, y_train,y_test = read_Dataset()

In [ ]:
# To compute the feature relevance in the models
relevant_vector = []

for i in range (100):
    model = load_model('./Models/model_'+str(i)+'.h5')
    dataset, y_dataset = read_T_A_Datasets(train_dataset,y_train,i)
    dataset.reset_index(drop = True,inplace = True)
    y_dataset.reset_index(drop = True,inplace = True)
    print('shape ', dataset.shape)
    print('shape', y_dataset.shape)
    print('y_train : ', y_dataset.value_counts())
    print('Counter : ', i)
    explainer = dx.Explainer(model, dataset, y_dataset)
    explanation = explainer.model_parts(random_state=42)
    variable_importance = pd.DataFrame(explanation.result)
    variable_importance = variable_importance.sort_values(by  = ['variable'], ascending= True)
    
    variable_importance.drop(['label'], axis=1, inplace = True)

    relevant_vector.append(variable_importance)
    

In [ ]:
### Feature Relevance
######### Append the Feature Relevance for the 100 models together #########################3

df = pd.DataFrame(relevant_vector[0].set_index(['variable']), columns=['variable', 'dropout_loss'])
df = df.rename(columns = {'dropout_loss':'0'})
df.drop(['variable'], axis=1, inplace=True)

df= df.transpose()

for i in range(len(relevant_vector)):
    if i==0:
        continue
    else:
        df1 = pd.DataFrame(relevant_vector[i].set_index(['variable']), columns = ['variable', 'dropout_loss'])
        df1.drop(['variable'], axis=1, inplace=True)
        df1 = df1.rename(columns = {'dropout_loss':str(i)})
        df1 = df1.transpose()
        df = df.append(df1)
    
    
dataset_path = 'C:/Users/Malik/PycharmProjects/Ensembled_for_Server/CIC_MalDroid_10%_470_features/Dalex/dalex_10%_100_CIC_MalDroid.csv'
df.to_csv(path_or_buf = dataset_path, index=False)


df

In [ ]:
# Save the file
path = 'C:/Users/Malik/PycharmProjects/Ensembled_for_Server/CIC_Maldroid_40_features/CIC_MalDroid2020_10%/Dalex/dalex_10%_100_CIC_MalDroid.csv'
df = pd.read_csv(path)

df.drop(['_full_model_'],axis = 1, inplace = True)
df.drop(['_baseline_'], axis = 1, inplace = True)
df

In [ ]:
# To the ELBOW

from sklearn_extra.cluster import KMedoids

from yellowbrick.cluster import KElbowVisualizer
kmedoids = KMedoids( init='k-medoids++', method = 'pam',random_state= 42)

visualizer = KElbowVisualizer(kmedoids, k=(100), metric='distortion')
visualizer.fit(df)        # Fit the data to the visualizer
visualizer.show()


In [ ]:
# The ELBOW will be used here to create the clusters
kmedoids = KMedoids(n_clusters = , init = 'k-medoids++', method = 'pam', random_state=42)#np.random.seed(seed))

kmedoid = kmedoids.fit(df)

In [ ]:
# Print the Medoids

medoid = kmedoid.medoid_indices_
medoid.sort()
print(medoid)
print(kmedoid.inertia_)
print(len(medoid))

In [ ]:
# to print the cluster's members
re = kmedoid.predict(df)
re

clusters = []
dictionary = {}
n = 0
for item in range(100):
    #print(item)
    if re[item] in dictionary:
            value = []
            value = dictionary[re[item]]
            value.append(item)
            dictionary[re[item]] = value
        
            #print(clusters[item])
    else:
            dictionary[re[item]] = [item]
           # print(clusters[re[item]])

sorted_clusters = sorted(dictionary.items(), key=lambda item: item[0])
#print(sorted_clusters)

for i in range(len(sorted_clusters)):
    print(sorted_clusters[i])